# 1 Knapsack Problem

The knapsack problem is one of the most trivial NP-hard problems. We can encounter many of its variants in the literature, which generally have various solution requirements for the algorithm. The formulation always contains criterium for the overall price, which means it is an optimization problem.

## 1.1 Base of Problem

We are given

* integer $n$ (number of items)
* integer $M$ (capacity of knapsack)
* finite set $V = \{v_1, v_2, \dots, v_n\}$ (weights of items)
* finite set $C = \{c_1, c_2, \dots, c_n\}$ (prices of items)


## 1.2 Decision 0/1 Problem Form

In this task, we will solve a decision problem form, which is one of the variants with some specific requirements:

In addition to the input variables we define

* positive integer $B$ (minimal required price)

We say that the problem has a solution if we can find a set $X=\{x_1, x_2, \dots, x_n\}$ where each $x_i$ is either $0$ or $1$, and

$v_1x_1 + v_2x_2 + \dots + v_nx_n \le M$ (knapsack is not overloaded), and

$c_1x_1 + c_2x_2 + \dots + c_nx_n \ge B$ (price is greater or equal to required price).

<br>

<cite data-cite="6752554/CXL66Z59"></cite>

# 2 Solution approaches


In the following section, we will describe two basic approaches (algorithms) to solve this problem:

* Brute force
* Branch and bound (B&B)

Both algorithms compute the optimal price and the configuration (vector of $n$-bits). The vector is equivalent to the set $X$ described in the **section 1.2**. We decide whether the problem is solvable by comparing the optimal price and the minimal required price $B$. The problem is solvable if the resulting price meets the requirements described in the **section 1.2**.

In both of the algorithms, we use recursion to search the state space.

## 2.1 Brute Force

In the brute force approach, we search (almost) the whole state space to find the optimal price and configuration. The truth is that we do not go through the entire state space – that's why almost. We add a simple condition that cuts the recursion branch if adding an item would result in the overloaded knapsack.

Let us look at the code snippet of the algorithm:

``` python
def brute_force(conf, i, weight, price):
    if i == instance.size:
        solution.complexity += 1
        if price >= solution.price:
            solution.price = price
            solution.weight = weight
            solution.conf = conf
        return

    new_weight = instance.items[i].weight + weight

    conf[i] = 1
    if new_weight <= instance.capacity:
        new_price = instance.items[i].price + price
        brute_force(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    brute_force(conf, i + 1, weight, price)
```

The algorithm starts by comparing the current depth of the recursion `i` with the size of the instance. We use to comparison to determine if we have already solved one of the branches of recursion. We increment the counter of the complexity – we later use this data in the analysis. If the `price` is greater or equal to the current optimal price, we update the `solution` object with the new values – `price`, `weight`, and vector configuration `conf`.

Next, we compute a new weight by adding the current weight to the particular items' weight. To cover the whole state space, we have to call the recursion with the current bit set to either $0$ or $1$. We also have to recalculate the price and weight. We later send them the recursion if the bit is set to $1$ – it means that the item is added to the knapsack. Also, as mentioned above, we call the recursion with the added item only if the recalculated weight is less than or equal to the capacity of the knapsack – we don't overload the knapsack.

The algorithm searches (almost) the entire state space and correctly computes the optimal price. The configuration, weight, and optimal price are stored in the `solution` object. We further use this objects' data in the **section 3** (analysis) and to answer the question of whether a particular instance is solvable or not.

## 2.2 Branch and Bound (B&B)

The beginning of the B&B algorithm is the same as in the brute force algorithm. The only difference is in the second condition for the cutting of the recursion calls (state space). Let us look at the difference in the algorithm:

``` python
def branch_and_bound(conf, i, weight, price):
    # The first condition is same as in the brute force algorithm...
    
    new_weight = instance.items[i].weight + weight
    new_price = instance.items[i].price + price
    upper_bound = price + instance.prices_sum(i=i)

    conf[i] = 1
    if (new_weight <= instance.capacity) and (upper_bound >= solution.price):
        branch_and_bound(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    branch_and_bound(conf, i + 1, weight, price)
```

We enhance the condition by checking if the current `price` added to the sum of prices of the unvisited items (on the index `i` and greater) is greater or equal to the current optimal price. We can cut the recursion branch if we do not meet this condition. This way, we cut the branches of the state space, which do not lead us to the optimal price.

# 3 Results Analysis


We analyze the data using the *Python* language and its mathematical modules. The graphs and the whole report are rendered in the *Jupyter Notebook* – that's why we also add the code snippets in this report, which process data and generate the graphs. Data generated by the solution code are stored in the CSV format files, so we can easily load and process them.

We start by loading and processing the data:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
data = {
    "NR": {"bb": {}, "bf": {}},
    "ZR": {"bb": {}, "bf": {}},
}

dtypes = {"solvable": bool, "conf": str, "price": int, "weight": int, "time": float, "complexity": int}

for v in ["NR", "ZR"]:
    for t in ["bb", "bf"]:
        for n in [4, 10, 15, 20, 22]:
            df = pd.read_csv(f"../data/output/{v}/{t}/{v}{n}_out.csv", sep=";", dtype=dtypes )
            data[v][t][n] = df

In [12]:
data["NR"]["bb"][4]

,solvable,conf,price,weight,time,complexity
0,True,0100,1129,43,0.000048,3
1,True,0110,2966,104,0.000022,5
2,True,0000,0,0,0.000008,1
3,True,1000,70,23,0.000013,2
4,False,0101,2758,223,0.000023,6
...,...,...,...,...,...,...
495,False,0111,3106,109,0.000014,4
496,True,1110,4426,240,0.000025,6
497,True,0001,2145,133,0.000008,2
498,True,0100,2036,22,0.000011,2


First, we look at the dependency between the size of the instance $n$ and the complexity of the computation. Complexity is defined as the number of the evaluated configurations of the $n$-bit vector. In the first graph, we take the average complexity of the instances:

# Bibliography

<br>

<div class="cite2c-biblio"></div>